In [1]:
%load_ext autoreload

In [14]:
import mysql.connector as mysql
import numpy as np
import base64
import PIL
from PIL import Image as Image
#from IPython.display import Image
#import urllib.parse
import io
import re
from app.src.db import *
%autoreload 2

In [15]:

mydb = mysql.connect(
    host="127.0.0.1",
    port=49157,
    user="root",
    password="datascience",
    database="racer")


In [16]:
dbcursor = mydb.cursor()


In [17]:
dbcursor.execute("SELECT * FROM images")
results = dbcursor.fetchall()


In [18]:
#createBatch(mydb, "alpha2", "Beginning CNN testing, starting with fresh data.")
getBatches(mydb)

[(5, 'alpha testing', 'Very unstable, no assumptions about consistency'),
 (6, 'alpha2', 'Beginning CNN testing, starting with fresh data.')]

In [19]:
getLayers(mydb)

[(1, 'imageDelta', 'currentframe minus the previous frame', 0, 5)]

In [282]:
#newlayerid = createProcessingLayer(mydb, 0, 5, "imageDelta", "currentframe minus the previous frame")

In [283]:
newlayerid

1

In [21]:
def processorDIFF(targetFrame, referenceFrame):
    return referenceFrame - targetFrame

def process(db, layer, batch, processor, newlayerid=None):
    dbcursor = mydb.cursor()
    dbcursor.execute(f"SELECT * FROM images WHERE layerid = '{layer}' and batchid = '{batch}'")
    lasttime=0
    lastimage_np = None
    results = dbcursor.fetchall()
    print(f"num rows to process: {len(results)}")
    saved_images = 0
    for result in results:
        imageuri = result[7]
        time = result[1]
        position = result[2]
        velocity = result[3]
        rotation = result[4]
        layer    = result[5]
        batch    = result[6]
        
        tdelta = time - lasttime
        lasttime = time
        
        image_np = uriToNP(imageuri)
        if lastimage_np is not None:
            image_diff = processor(lastimage_np, image_np)
            a = np.uint8(image_diff)
            f = io.StringIO()
            newimage = PIL.Image.fromarray(a)
            
            if newlayerid is not None:
                uri_image = pilToURI(newimage)
                
                #display(newimage)
                #showURI(uri_image)
                #print(uri_image)
                try:
                    saveImage(db, batch, newlayerid, time, position, velocity, rotation, uri_image)
                    saved_images += 1
                except Exception as ex:
                    print(result[0])
                    print("Exception: " + ex)
        lastimage_np = image_np
        
    print(f"saved {saved_images} images into layer {newlayerid} batch {batch}")

In [22]:
process(mydb, 0, 6, processorDIFF, 1)

num rows to process: 3295
saved 3294 images into layer 1 batch 6


In [293]:
def makeVideo(db, layer, batch):
    #this function is broken, use makeGif and then convert the resulting gif into a webm
    import cv2
    import os

    video_name = 'video.avi'
    dbcursor = mydb.cursor()
    dbcursor.execute(f"SELECT * FROM images WHERE layerid = '{layer}' and batchid = '{batch}'")
    results = dbcursor.fetchall()
    #images = [img for img in os.listdir(image_folder) if img.endswith(".png")]
    #frame = cv2.imread(os.path.join(image_folder, images[0]))
    height, width, layers = 50, 50, len(results)
    fourcc = cv2.VideoWriter_fourcc(*'MP4V')
    video = cv2.VideoWriter(video_name, 0x7634706d, 1, (300,300))
    for result in results:
        #print(result[7])
        image = uriToNP2(result[7])
        cv2.imshow('frame',image)
        video.write(image)

    cv2.destroyAllWindows()
    video.release()

def makeGif(db, filename, layer, batch):
    
    import imageio
    
    dbcursor = mydb.cursor()
    dbcursor.execute(f"SELECT * FROM images WHERE layerid = '{layer}' and batchid = '{batch}'")
    results = dbcursor.fetchall()
    #images = [img for img in os.listdir(image_folder) if img.endswith(".png")]
    #frame = cv2.imread(os.path.join(image_folder, images[0]))
    height, width, layers = 50, 50, len(results)
    with imageio.get_writer(filename, mode='I') as writer:
        for result in results:
        #print(result[7])
            image = uriToNP(result[7])
            writer.append_data(image)
        #cv2.imshow('frame',image)
        #video.write(image)

    


def uriToNPXX(imageuri):
    imgstr = re.search(r'base64,(.*)', imageuri).group(1)
    image_bytes = io.BytesIO(base64.b64decode(imgstr))
    im = Image.open(image_bytes)
    nparray = np.array(im)
    if len(nparray.shape) == 2:
        arr = np.array(im)[:,:]
    elif len(nparray.shape) == 3:
        arr = np.array(im)[:,:,0]
    return arr

In [292]:
makeGif(mydb, "original.gif",0,5)

In [24]:
def getDataset(db, batch, layer):
    dbcursor = mydb.cursor()
    dbcursor.execute(f"SELECT * FROM images WHERE layerid = '{layer}' and batchid = '{batch}'")
    results = dbcursor.fetchall()
    X = None
    Y = None
    for result in results:
        image_arr = uriToNP(result[7])
        x_row = image_arr.reshape(1,-1)
        #print(x_row.shape)
        
        position = result[2]
        velocity = result[3]
        rotation = result[4]
        y_row = np.array([position, velocity, rotation]).reshape(1,-1)
        if X is not None:
            X = np.vstack((X, x_row))
            Y = np.vstack((Y, y_row))
        else:
            X = x_row
            Y = y_row
            
    return X, Y

In [25]:
X, Y = getDataset(mydb, 6, 1)


In [26]:
print(X.shape)
print(Y.shape)

(3294, 2500)
(3294, 3)


In [92]:
arr

array([[147, 147, 143, ..., 178, 176, 176],
       [142, 139, 138, ..., 181, 180, 178],
       [135, 135, 131, ..., 185, 183, 182],
       ...,
       [178, 178, 178, ..., 178, 178, 178],
       [178, 178, 178, ..., 178, 178, 178],
       [178, 178, 178, ..., 178, 178, 178]], dtype=uint8)

In [93]:
arr.shape

(50, 50)

In [56]:
data.shape

()

In [57]:
data

array(<IPython.core.display.Image object>, dtype=object)